In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os


transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = 'horse-or-human/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch #numworks가 뭐지?
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1027, shuffle=False, num_workers=0)  


validation_data_path = 'horse-or-human/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=256, shuffle=False, num_workers=0)  

NUM_EPOCH=1
for epoch in range(NUM_EPOCH):
    # load training images of the batch size for every iteration
    for i, data in enumerate(trainloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)  





    # load validation images of the batch size for every iteration
    for i, data in enumerate(valloader):
        
        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)    

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import numpy as np
import plotly.graph_objects as go

In [3]:
train_dataset = next(iter(trainloader))[0].numpy()
train_label =next(iter(trainloader))[1].numpy()

val_dataset = next(iter(valloader))[0].numpy()
val_label =next(iter(valloader))[1].numpy()

print(train_dataset.shape)

(1027, 1, 100, 100)


In [4]:
train_label=train_label.reshape(1027,1)
val_label=val_label.reshape(256,1)

In [5]:
train_flatten=train_dataset.reshape(1027,train_dataset.shape[2]*train_dataset.shape[3])
val_flatten=val_dataset.reshape(256,val_dataset.shape[2]*val_dataset.shape[3])

In [6]:
x_train=train_flatten.T
y_train=train_label.T
print(x_train.shape)
print(y_train.shape)

x_test=val_flatten.T
y_test=val_label.T
print(x_test.shape)
print(y_test.shape)

(10000, 1027)
(1, 1027)
(10000, 256)
(1, 256)


In [98]:
n_x = x_train.shape[0] # size of input layer`
n_h = 4 # size of hidden layer
n_y = y_train.shape[0] # size of output layer
print(n_x,n_y)
m = x_train.shape[1]

10000 1


In [123]:
learning_rate=0.01

W1 = np.random.randn(n_h,n_x) * 0.01
b1 = np.zeros(shape=(n_h, 1))

W2 = np.random.randn(n_h,n_h) * 0.01
b2 = np.zeros(shape=(n_h, 1))

W3 = np.random.randn(n_y,n_h) * 0.01
b3 = np.zeros(shape=(n_y, 1))
#parameters={W1, b1, W2, b2, W3, b3}
    #return {'W1': W1,'b1':b1, 'W2':W2, 'b2':b2, 'W3':W3, 'b3':b3 }
    #return W1, b1, W2, b2, W3, b3
    #, W1=0, b1=0, W2=0, b2=0, W3=0, b3=0

In [101]:
def sigmoid(z):
    y = 1/(1+np.exp(-z))
    return y

def relu(x):
    return np.maximum(0, x)


In [129]:
def forward_p(x_train):
# Implement Forward Propagation to calculate A2 (probabilities)
    #print(parameters)
    Z1 = np.dot(W1,x_train) + b1
    A1 = relu(Z1) # activation function
    
    Z2 = np.dot(W2,A1) + b2
    A2 = relu(Z2)  #np.tanh 바꾼거
    
    Z3 = np.dot(W3,A2) + b3
    A3 = sigmoid(Z3) # Final output prediction
    return A3, A2 , A1

In [114]:
x_test.shape[1]

256

In [71]:
# Compute the cross-entropy cost
def compute_cost(A3, Y):
    loss = np.multiply(np.log(A3), Y) + np.multiply((1 - Y), np.log(1 - A3))
    cost = - np.sum(loss) / m
    return cost

In [131]:
def back_p( X, Y, A3, A2, A1) :
    dZ3 = A3 - Y
    dW3 = (1 / m) * np.dot(dZ3, A2.T)
    db3 = (1 / m) * np.sum(dZ3, axis=1, keepdims=True)
    
    dZ2 = np.multiply(np.dot(W3.T, dZ3), 1 - np.power(A2, 2))
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    return {'dW1':dW1, 'db1':db1, 'dW2':dW2, 'db2':db2, 'dW3':dW3,'db3':db3}
    return dW1,db1,dW2,db2,dW3,db3

In [124]:
def update_parameters(  dW1=0,db1=0,dW2=0,db2=0,dW3=0,db3=0) :
    global W1,b1,W2,b2,W3,b3,learning_rate
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W3 = W3 - learning_rate * dW3
    b3 = b3 - learning_rate * db3

In [149]:
def predict( X) :
    A3,_,__ = forward_p(X)
    predictions = A3 >0.5
    
    return predictions
def print_accuracy(X, Y) :
    predictions = predict( X)
    print('Accuracy: %d' %(float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T)))/float(Y.size)*100)+ '%')

In [150]:
def nn_model(X, Y, n_h, num_iterations):
    #parameters = {'W1', 'b1','W2','b2','W3','b3'}
    #parameters = init_parameters(n_x,n_h,n_y) 
    #parameters = np.array(parameters)
    #parameters = parameters.reshape(6,1)
    #print('parameters', parameters.shape)
   # print(parameters)
    #W1 = parameters['W1']
    #b1 = parameters['b1']
    #W2 = parameters['W2']
    #b2 = parameters['b2']
    #W3 = parameters['W3']
    #b3 = parameters['b3']
    
    for i in range(0, num_iterations) :
        A3 ,A2, A1= forward_p(X)
        cost = compute_cost(A3, Y)
        grads = back_p( X, Y, A3, A2, A1)
        parameters = update_parameters( **grads)
        
        if i % 100 == 0:
            print("Cost after iteration %i : %f" %(i,cost))
            print_accuracy(X,Y)
            
    return parameters

In [151]:
parameters = nn_model(x_train, y_train ,n_h, 5000)

Cost after iteration 0 : 0.692823
Accuracy: 51%
Cost after iteration 100 : 0.692813
Accuracy: 51%
Cost after iteration 200 : 0.692807
Accuracy: 51%
Cost after iteration 300 : 0.692803
Accuracy: 51%
Cost after iteration 400 : 0.692801
Accuracy: 51%
Cost after iteration 500 : 0.692799
Accuracy: 51%
Cost after iteration 600 : 0.692798
Accuracy: 51%
Cost after iteration 700 : 0.692797
Accuracy: 51%
Cost after iteration 800 : 0.692796
Accuracy: 51%
Cost after iteration 900 : 0.692796
Accuracy: 51%
Cost after iteration 1000 : 0.692795
Accuracy: 51%
Cost after iteration 1100 : 0.692795
Accuracy: 51%
Cost after iteration 1200 : 0.692794
Accuracy: 51%
Cost after iteration 1300 : 0.692794
Accuracy: 51%
Cost after iteration 1400 : 0.692794
Accuracy: 51%


KeyboardInterrupt: 